In [2]:
import langchain

This Module Covers \n
--> Ingesting Data + Parsing = RAG Systems
1 . Intro to Data Ingestion
2 . TextFiles(.txt)
3 . MarkDown Files (.md)
4 . PDF Documents
5 . Microsoft Docs
6 . CSV/Excel Files
7 . JSON & Structured Data
8 . Web Scraping
9 . DataBases(SQL)
10 . Audio & Video Transcripts


In [3]:
import os
from typing import List, Dict, Any
import pandas as pd

In [4]:
from langchain_core.documents import Document
# Types of TextSlitting Techniques --> Chunking Process
from langchain_text_splitters import (RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)
print("Set up complete")

Set up complete


Understanding Document Struture in LangChain

In [5]:
## CREATE a simple document
doc = Document(
    page_content="tHIS IS rag CONTENT",
    metadata={
        "source": "example.txt",
        "page":1,
        "author": "ABC",
        "date_created": "2025-12-12",
    } 
)
print("Document Structure")
print(f"Content: {doc.page_content}")
print(f"Content: {doc.metadata}")

Document Structure
Content: tHIS IS rag CONTENT
Content: {'source': 'example.txt', 'page': 1, 'author': 'ABC', 'date_created': '2025-12-12'}


In [6]:
type(doc)

langchain_core.documents.base.Document

In [7]:
#Create a simple Text File: 
import os
os.makedirs("data/text_files", exist_ok=True)

In [8]:
sample_texts = {
    "data/text_files/python_intro.txt": """Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems

    """ 
}

for (filepath,content) in sample_texts.items():
    with open(filepath, 'w', encoding="utf-8") as f:
        f.write(content)

print("sample file is created") 

sample file is created


How are you going to read this library ?
#TEXT LOADERRR

In [9]:
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

loader=TextLoader("data/text_files/python_intro.txt", encoding="utf-8")

documents = loader.load()

print(f"Loaded {len(documents)} document")
print(f"Content peview: {documents[0].page_content}")
print(f"MetaData: {documents[0].metadata}")

Loaded 1 document
Content peview: Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.
MetaData: {'source': 'data/text_files/python_intro.txt'}


DirectoryLoader - Multiple Text Files

In [10]:
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "data/text_files",
    glob="*.txt", ##Patern to match files
    loader_cls= TextLoader, ##loader class to use
    loader_kwargs={'encoding':'utf-8'},
    show_progress=True
)

documentss = dir_loader.load()
print(f"Loaded {len(documentss)} documents")
for i,doc in enumerate(documentss):
    print(f"\nDocument {i+1}")
    print(f"  Source: {doc.metadata['source']}")
    print(f"  Length: {len(doc.page_content)} characters")

100%|██████████| 2/2 [00:00<00:00, 606.60it/s]

Loaded 2 documents

Document 1
  Source: data\text_files\machine_learning.txt
  Length: 574 characters

Document 2
  Source: data\text_files\python_intro.txt
  Length: 489 characters


Text Splitting 

In [11]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter
)
print(documents)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [12]:
#Method 1 --> CharacterTextSplitter
text=documents[0].page_content
text

'Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'

In [13]:
print("Character Text Splitter")
char_splitter = CharacterTextSplitter(
    separator="\n", #Split on new lines
    chunk_size=200, #Max chunksize in characters
    chunk_overlap=20, #Overlap b/w chunks
    length_function=len #How to measure chunk size
)

char_chunks=char_splitter.split_text(text)
print(f"Creted {len(char_chunks)} chunks")
print(f"First Chunk: {char_chunks[0][:100]}...")

Character Text Splitter
Creted 3 chunks
First Chunk: Python Programming Introduction
Python is a high-level, interpreted programming language known for i...


In [14]:
print(char_chunks[0])
print("---------------")
print(char_chunks[1])
print("--------------")
print(char_chunks[2])

Python Programming Introduction
Python is a high-level, interpreted programming language known for its simplicity and readability.
---------------
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.
Key Features:
- Easy to learn and use
- Extensive standard library
--------------
- Cross-platform compatibility
- Strong community support
Python is widely used in web development, data science, artificial intelligence, and automation.


In [15]:
#Method 2: Recursive character Splitting (RECOMMENDED)
print(f"\nRECURSIVE CHARCTER TEXT SPLITTER")
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""], #Try these separators in order
    chunk_size=200,
    chunk_overlap=20,
    length_function=len
)


RECURSIVE CHARCTER TEXT SPLITTER


In [18]:
# Method 3: Token-Based text splitting
print("\n TOKEN TEXT SPLITTING")
token_splitter=TokenTextSplitter(
    chunk_size=50, # size in tokens (not characters)
    chunk_overlap=10 
)

total_chunks=token_splitter.split_text(text)
print(f"Created {len(total_chunks)} chunks")
print(f"First Chunk: {total_chunks[0]}...")



 TOKEN TEXT SPLITTING
Created 3 chunks
First Chunk: Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in...
